# English Premier League (EPL) Pythagorean Predictor

## Step 1

In [13]:
# Load the packages

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
# Load the data. 
# EPL results for the 2017/18 season
path = r"D:\Sports\Assignments\Assignment Data\Week 1\EPL2017-18.xlsx"
EPL18 =  pd.read_excel(path, engine='openpyxl')
print(EPL18.columns.tolist())


['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']


In [15]:
EPL18.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,20170811,Arsenal,Leicester,4,3,H
1,20170812,Brighton,Man City,0,2,A
2,20170812,Chelsea,Burnley,2,3,A
3,20170812,Crystal Palace,Huddersfield,0,3,A
4,20170812,Everton,Stoke,1,0,H


In [16]:
EPL18.rename(columns={
    'Date': 'Match_Date',
    'HomeTeam': 'Home_Team',
    'AwayTeam': 'Away_Team',
    'FTHG': 'Home_Team_Goals',
    'FTAG': 'Away_Team_Goals',
    'FTR': 'Match_Result'
} , inplace= True)

In [17]:
EPL18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Match_Date       380 non-null    int64 
 1   Home_Team        380 non-null    object
 2   Away_Team        380 non-null    object
 3   Home_Team_Goals  380 non-null    int64 
 4   Away_Team_Goals  380 non-null    int64 
 5   Match_Result     380 non-null    object
dtypes: int64(3), object(3)
memory usage: 17.9+ KB


In [18]:
EPL18.isna().any()

Match_Date         False
Home_Team          False
Away_Team          False
Home_Team_Goals    False
Away_Team_Goals    False
Match_Result       False
dtype: bool

## Step 2
Create a value for a home wins (win= 1, draw=0.5, loss= 0) and away wins and a count variable for each game (=1).



In [19]:
EPL18['Home_Win'] = EPL18.apply(
    lambda row: 1 if row['Home_Team_Goals'] > row['Away_Team_Goals'] 
   else 0, axis=1
   )

EPL18['Away_Win'] = EPL18.apply(
    lambda row: 1 if row['Away_Team_Goals'] > row['Home_Team_Goals'] else 0, axis=1 
    )

EPL18['Draw'] = EPL18.apply(
    lambda row: 1 if row['Home_Team_Goals'] == row['Away_Team_Goals'] else 0, axis=1
    )
EPL18['Game_Count'] = 1  # each game is counted as 1


In [20]:
EPL18

,Match_Date,Home_Team,Away_Team,Home_Team_Goals,Away_Team_Goals,Match_Result,Home_Win,Away_Win,Draw,Game_Count
0,20170811,Arsenal,Leicester,4,3,H,1,0,0,1
1,20170812,Brighton,Man City,0,2,A,0,1,0,1
2,20170812,Chelsea,Burnley,2,3,A,0,1,0,1
3,20170812,Crystal Palace,Huddersfield,0,3,A,0,1,0,1
4,20170812,Everton,Stoke,1,0,H,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
375,20180513,Newcastle,Chelsea,3,0,H,1,0,0,1
376,20180513,Southampton,Man City,0,1,A,0,1,0,1
377,20180513,Swansea,Stoke,1,2,A,0,1,0,1
378,20180513,Tottenham,Leicester,5,4,H,1,0,0,1


## Step 3
Create a file for games played in 2017 (before date 20180000) and another one for games played in 2018 (after date 20180000).



In [21]:
EPL_2017 = EPL18[EPL18['Match_Date'] < 20180000]

# Games played in 2018 (i.e., after 20180000)
EPL_2018 = EPL18[EPL18['Match_Date'] >= 20180000]

In [22]:
EPL_2018

,Match_Date,Home_Team,Away_Team,Home_Team_Goals,Away_Team_Goals,Match_Result,Home_Win,Away_Win,Draw,Game_Count
209,20180101,Brighton,Bournemouth,2,2,D,0,0,1,1
210,20180101,Burnley,Liverpool,1,2,A,0,1,0,1
211,20180101,Everton,Man United,0,2,A,0,1,0,1
212,20180101,Leicester,Huddersfield,3,0,H,1,0,0,1
213,20180101,Stoke,Newcastle,0,1,A,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
375,20180513,Newcastle,Chelsea,3,0,H,1,0,0,1
376,20180513,Southampton,Man City,0,1,A,0,1,0,1
377,20180513,Swansea,Stoke,1,2,A,0,1,0,1
378,20180513,Tottenham,Leicester,5,4,H,1,0,0,1


## Step 4 (home team)
For the 2017 games, use .groupby to create a dataframe aggregating by home team the variables for count, home wins, goals for and goals against.



In [23]:
home_stats = EPL_2017.groupby('Home_Team')[['Game_Count', 'Home_Win','Home_Team_Goals','Away_Team_Goals']].sum().reset_index()

In [24]:
home_stats18 = EPL_2018.groupby('Home_Team')[['Game_Count', 'Home_Win','Home_Team_Goals','Away_Team_Goals']].sum().reset_index()

In [25]:
home_stats.rename(columns= { 
                             'Home_Win': 'Home_Wins',
                             'Home_Team_Goals': 'Goals_for',
                             'Away_Team_Goals' : 'Goals_against'} , inplace= True)


In [26]:
home_stats18.rename(columns= { 
                             'Home_Win': 'Home_Wins',
                             'Home_Team_Goals': 'Goals_for',
                             'Away_Team_Goals' : 'Goals_against'} , inplace= True)


In [27]:
home_stats

,Home_Team,Game_Count,Home_Wins,Goals_for,Goals_against
0,Arsenal,10,8,25,10
1,Bournemouth,11,3,14,17
2,Brighton,10,3,10,12
3,Burnley,10,5,7,6
4,Chelsea,11,8,21,7
5,Crystal Palace,11,3,14,18
6,Everton,10,6,17,13
7,Huddersfield,11,4,10,12
8,Leicester,10,4,13,14
9,Liverpool,11,6,21,4


In [28]:
home_stats18

,Home_Team,Game_Count,Home_Wins,Goals_for,Goals_against
0,Arsenal,9,7,29,10
1,Bournemouth,8,4,12,13
2,Brighton,9,4,14,13
3,Burnley,9,2,9,11
4,Chelsea,8,3,9,9
5,Crystal Palace,8,4,15,9
6,Everton,9,4,11,9
7,Huddersfield,8,2,6,13
8,Leicester,9,3,12,8
9,Liverpool,8,6,24,6


## Step 5 (home team)
Then, use .groupby to  create a separate dataframe aggregating by away team the variables for count, away wins, goals for and goals against.



In [29]:
away_stats = EPL_2017.groupby('Away_Team')[['Game_Count', 'Away_Win','Away_Team_Goals', 'Home_Team_Goals']].sum().reset_index()


# Rename columns for better clarity
away_stats = away_stats.rename(columns={
    'Away_Team': 'Away_Team_Name',
    'Away_Win': 'Away_Wins',
    'Home_Team_Goals': 'Goals_against',
    'Away_Team_Goals': 'Goals_for'
    
})

# Display the result
away_stats

,Away_Team_Name,Game_Count,Away_Wins,Goals_for,Goals_against
0,Arsenal,11,3,13,16
1,Bournemouth,10,2,6,15
2,Brighton,11,2,5,13
3,Burnley,11,4,11,11
4,Chelsea,10,6,18,7
5,Crystal Palace,10,1,4,14
6,Everton,11,1,8,19
7,Huddersfield,10,2,8,20
8,Leicester,11,3,18,18
9,Liverpool,10,5,27,20


In [30]:
away_stats18 = EPL_2018.groupby('Away_Team')[['Game_Count', 'Away_Win','Away_Team_Goals', 'Home_Team_Goals']].sum().reset_index()


# Rename columns for better clarity
away_stats18 = away_stats18.rename(columns={
    'Away_Team': 'Away_Team_Name',
    'Away_Win': 'Away_Wins',
    'Home_Team_Goals': 'Goals_against',
    'Away_Team_Goals': 'Goals_for'
    
})

# Display the result
away_stats18

,Away_Team_Name,Game_Count,Away_Wins,Goals_for,Goals_against
0,Arsenal,8,1,7,15
1,Bournemouth,9,2,13,16
2,Brighton,8,0,5,16
3,Burnley,8,3,9,11
4,Chelsea,9,4,14,15
5,Crystal Palace,9,3,12,14
6,Everton,8,2,8,17
7,Huddersfield,9,1,4,13
8,Leicester,8,2,13,20
9,Liverpool,9,4,12,8


## Step 6 (home team)

In [31]:
away_stats = away_stats.rename(columns={
    "Game_Count": 'Away_Game_Count',
    'Away_Team_Name': 'Away_Team_Name',
    'Away_Wins': 'Away_Wins',
    'Goals_against': 'Away_Goals_Conceded',
    'Goals_for': 'Away_Goals_Scored'
})

home_stats.rename(columns={
    "Game_Count": "Home_Game_Count",
    'Home_Wins': 'Home_Wins',
    'Goals_for': 'Home_Goals_Scored',
    'Goals_against': 'Home_Goals_Conceded'
}, inplace=True)




In [32]:
away_stats18 = away_stats18.rename(columns={
    "Game_Count": 'Away_Game_Count',
    'Away_Team_Name': 'Away_Team_Name',
    'Away_Wins': 'Away_Wins',
    'Goals_against': 'Away_Goals_Conceded',
    'Goals_for': 'Away_Goals_Scored'
})

home_stats18.rename(columns={
    "Game_Count": "Home_Game_Count",
    'Home_Wins': 'Home_Wins',
    'Goals_for': 'Home_Goals_Scored',
    'Goals_against': 'Home_Goals_Conceded'
}, inplace=True)




In [33]:
home_stats.head()

,Home_Team,Home_Game_Count,Home_Wins,Home_Goals_Scored,Home_Goals_Conceded
0,Arsenal,10,8,25,10
1,Bournemouth,11,3,14,17
2,Brighton,10,3,10,12
3,Burnley,10,5,7,6
4,Chelsea,11,8,21,7


In [34]:
home_stats18.head()

,Home_Team,Home_Game_Count,Home_Wins,Home_Goals_Scored,Home_Goals_Conceded
0,Arsenal,9,7,29,10
1,Bournemouth,8,4,12,13
2,Brighton,9,4,14,13
3,Burnley,9,2,9,11
4,Chelsea,8,3,9,9


In [35]:
# Merging the home and away stats dataframes on the team column
full_stats17 = pd.merge(home_stats, away_stats,  left_on="Home_Team", right_on="Away_Team_Name")

full_stats17.head()


,Home_Team,Home_Game_Count,Home_Wins,Home_Goals_Scored,Home_Goals_Conceded,Away_Team_Name,Away_Game_Count,Away_Wins,Away_Goals_Scored,Away_Goals_Conceded
0,Arsenal,10,8,25,10,Arsenal,11,3,13,16
1,Bournemouth,11,3,14,17,Bournemouth,10,2,6,15
2,Brighton,10,3,10,12,Brighton,11,2,5,13
3,Burnley,10,5,7,6,Burnley,11,4,11,11
4,Chelsea,11,8,21,7,Chelsea,10,6,18,7


In [36]:
# Merging the home and away stats dataframes on the team column
full_stats18 = pd.merge(home_stats18, away_stats18,  left_on="Home_Team", right_on="Away_Team_Name")

full_stats18.head()



,Home_Team,Home_Game_Count,Home_Wins,Home_Goals_Scored,Home_Goals_Conceded,Away_Team_Name,Away_Game_Count,Away_Wins,Away_Goals_Scored,Away_Goals_Conceded
0,Arsenal,9,7,29,10,Arsenal,8,1,7,15
1,Bournemouth,8,4,12,13,Bournemouth,9,2,13,16
2,Brighton,9,4,14,13,Brighton,8,0,5,16
3,Burnley,9,2,9,11,Burnley,8,3,9,11
4,Chelsea,8,3,9,9,Chelsea,9,4,14,15


In [37]:
full_stats18

,Home_Team,Home_Game_Count,Home_Wins,Home_Goals_Scored,Home_Goals_Conceded,Away_Team_Name,Away_Game_Count,Away_Wins,Away_Goals_Scored,Away_Goals_Conceded
0,Arsenal,9,7,29,10,Arsenal,8,1,7,15
1,Bournemouth,8,4,12,13,Bournemouth,9,2,13,16
2,Brighton,9,4,14,13,Brighton,8,0,5,16
3,Burnley,9,2,9,11,Burnley,8,3,9,11
4,Chelsea,8,3,9,9,Chelsea,9,4,14,15
5,Crystal Palace,8,4,15,9,Crystal Palace,9,3,12,14
6,Everton,9,4,11,9,Everton,8,2,8,17
7,Huddersfield,8,2,6,13,Huddersfield,9,1,4,13
8,Leicester,9,3,12,8,Leicester,8,2,13,20
9,Liverpool,8,6,24,6,Liverpool,9,4,12,8


In [38]:
#full_stats_2017_2018 = pd.merge(full_stats17, full_stats18, on="Home_Team",suffixes=['_2017', '_2018'])

In [39]:
#full_stats_2017_2018

## Optional steps, not required for Assessment 
### (Uncomment to run)

In [40]:
# Plot the data

#sns.relplot(x="pyth17", y="wpc17", data =EPL17)

In [41]:
# Run the regression

#pyth_lm = smf.ols(formula = 'wpc17 ~ pyth17', data=EPL17).fit()
#pyth_lm.summary()

## Step 7 (=Step 4 (away team))

## Step 7 (=Step 5 (away team))

## Step 7 (=Step 6 (away team))

## Optional steps, not required for Assessment
### (Uncomment to run)

In [42]:
# Plot the data

#sns.relplot(x="pyth18", y="wpc18", data =EPL2018)

In [43]:
# Run the regression

#pyth_lm = smf.ols(formula = 'wpc18 ~ pyth18', data=EPL2018).fit()
#pyth_lm.summary()

In [44]:
full_stats17.head()

,Home_Team,Home_Game_Count,Home_Wins,Home_Goals_Scored,Home_Goals_Conceded,Away_Team_Name,Away_Game_Count,Away_Wins,Away_Goals_Scored,Away_Goals_Conceded
0,Arsenal,10,8,25,10,Arsenal,11,3,13,16
1,Bournemouth,11,3,14,17,Bournemouth,10,2,6,15
2,Brighton,10,3,10,12,Brighton,11,2,5,13
3,Burnley,10,5,7,6,Burnley,11,4,11,11
4,Chelsea,11,8,21,7,Chelsea,10,6,18,7


## Step 8

In [45]:
full_stats17['Total_Wins'] = full_stats17['Home_Wins'] + full_stats17['Away_Wins']
full_stats17['Total_Games'] = full_stats17['Home_Game_Count'] + full_stats17['Away_Game_Count']
full_stats17['Total_Goals_For'] = full_stats17['Home_Goals_Scored'] + full_stats17['Away_Goals_Scored']
full_stats17['Total_Goals_Against'] = full_stats17['Home_Goals_Conceded'] + full_stats17['Away_Goals_Conceded']


In [46]:
full_stats18['Total_Wins'] = full_stats18['Home_Wins'] + full_stats18['Away_Wins']
full_stats18['Total_Games'] = full_stats18['Home_Game_Count'] + full_stats18['Away_Game_Count']
full_stats18['Total_Goals_For'] = full_stats18['Home_Goals_Scored'] + full_stats18['Away_Goals_Scored']
full_stats18['Total_Goals_Against'] = full_stats18['Home_Goals_Conceded'] + full_stats18['Away_Goals_Conceded']


In [47]:
full_stats18

,Home_Team,Home_Game_Count,Home_Wins,Home_Goals_Scored,Home_Goals_Conceded,Away_Team_Name,Away_Game_Count,Away_Wins,Away_Goals_Scored,Away_Goals_Conceded,Total_Wins,Total_Games,Total_Goals_For,Total_Goals_Against
0,Arsenal,9,7,29,10,Arsenal,8,1,7,15,8,17,36,25
1,Bournemouth,8,4,12,13,Bournemouth,9,2,13,16,6,17,25,29
2,Brighton,9,4,14,13,Brighton,8,0,5,16,4,17,19,29
3,Burnley,9,2,9,11,Burnley,8,3,9,11,5,17,18,22
4,Chelsea,8,3,9,9,Chelsea,9,4,14,15,7,17,23,24
5,Crystal Palace,8,4,15,9,Crystal Palace,9,3,12,14,7,17,27,23
6,Everton,9,4,11,9,Everton,8,2,8,17,6,17,19,26
7,Huddersfield,8,2,6,13,Huddersfield,9,1,4,13,3,17,10,26
8,Leicester,9,3,12,8,Leicester,8,2,13,20,5,17,25,28
9,Liverpool,8,6,24,6,Liverpool,9,4,12,8,10,17,36,14


In [48]:
full_stats17['wpc'] = full_stats17['Total_Wins'] / full_stats17['Total_Games']

full_stats17['pyth'] = (full_stats17['Total_Goals_For']**2) / (full_stats17['Total_Goals_For']**2 + full_stats17['Total_Goals_Against']**2)

In [49]:
full_stats18['wpc'] = full_stats18['Total_Wins'] / full_stats18['Total_Games']

full_stats18['pyth'] = (full_stats18['Total_Goals_For']**2) / (full_stats18['Total_Goals_For']**2 + full_stats18['Total_Goals_Against']**2)

In [50]:
full_stats17.drop("Away_Team_Name" , inplace = True , axis = 1)
full_stats17.rename(columns= {"Home_Team": "Team_Name"}, inplace= True )


In [51]:
full_stats18.drop("Away_Team_Name" , inplace = True , axis = 1)
full_stats18.rename(columns= {"Home_Team": "Team_Name"}, inplace= True )


In [52]:
full_stats17.head()

,Team_Name,Home_Game_Count,Home_Wins,Home_Goals_Scored,Home_Goals_Conceded,Away_Game_Count,Away_Wins,Away_Goals_Scored,Away_Goals_Conceded,Total_Wins,Total_Games,Total_Goals_For,Total_Goals_Against,wpc,pyth
0,Arsenal,10,8,25,10,11,3,13,16,11,21,38,26,0.523810,0.681132
1,Bournemouth,11,3,14,17,10,2,6,15,5,21,20,32,0.238095,0.280899
2,Brighton,10,3,10,12,11,2,5,13,5,21,15,25,0.238095,0.264706
3,Burnley,10,5,7,6,11,4,11,11,9,21,18,17,0.428571,0.528548
4,Chelsea,11,8,21,7,10,6,18,7,14,21,39,14,0.666667,0.885847


In [53]:
full_stats18.head()

,Team_Name,Home_Game_Count,Home_Wins,Home_Goals_Scored,Home_Goals_Conceded,Away_Game_Count,Away_Wins,Away_Goals_Scored,Away_Goals_Conceded,Total_Wins,Total_Games,Total_Goals_For,Total_Goals_Against,wpc,pyth
0,Arsenal,9,7,29,10,8,1,7,15,8,17,36,25,0.470588,0.674649
1,Bournemouth,8,4,12,13,9,2,13,16,6,17,25,29,0.352941,0.426330
2,Brighton,9,4,14,13,8,0,5,16,4,17,19,29,0.235294,0.300333
3,Burnley,9,2,9,11,8,3,9,11,5,17,18,22,0.294118,0.400990
4,Chelsea,8,3,9,9,9,4,14,15,7,17,23,24,0.411765,0.478733


## Step 9

## Optional steps, not required for Assessment
### (Uncomment to run)

In [54]:
#sns.relplot(x="pyth17", y="wpc18", data =Half2predictor)

In [55]:
#sns.relplot(x="wpc17", y="wpc18", data =Half2predictor)